In [ ]:
import csv
import hashlib
import requests

API_KEY = '1Fg2ES1FPAdPb79jhjAwkFAUtd5jV5bIkfOpCoUN'


def display_near_earth_objects():
    try:
        url = 'https://api.nasa.gov/neo/rest/v1/feed'
        params = {
            'api_key': API_KEY,
        }

        response = requests.get(url, params=params)
        response.raise_for_status() 
        data = response.json()

       
        near_earth_objects = data.get('near_earth_objects', {})

        if not near_earth_objects:
            print("No near-Earth objects found in the given date range.")
            return

       
        for date, neos in near_earth_objects.items():
            print(f"\nClose approach date: {date}")
            for neo in neos:
                name = neo.get('name', 'Unknown')
                diameter_min = neo['estimated_diameter']['meters']['estimated_diameter_min']
                diameter_max = neo['estimated_diameter']['meters']['estimated_diameter_max']
                velocity = neo['close_approach_data'][0]['relative_velocity']['kilometers_per_hour']
                miss_distance = neo['close_approach_data'][0]['miss_distance']['kilometers']
                hazardous = neo.get('is_potentially_hazardous_asteroid', False)

                print(f"Name: {name}")
                print(f"Estimated Diameter (m): {diameter_min:.2f} - {diameter_max:.2f}")
                print(f"Velocity (km/h): {velocity}")
                print(f"Miss Distance (km): {miss_distance}")
                print(f"Hazardous: {'Yes' if hazardous else 'No'}")
                
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching Near Earth Objects: {e}")
    except KeyError as e:
        print(f"Missing expected data in response: {e}")


def display_solar_object_info():
    object_id = input("Enter the name or ID of the Solar System object (e.g., Ceres, Mars): ")
    if not object_id:
        print("Object ID cannot be empty.")
        return

    try:
        url = f'https://ssd-api.jpl.nasa.gov/sbdb.api?sstr={object_id}'
        
        response = requests.get(url)
        response.raise_for_status()  
        data = response.json()

        if 'message' in data:
            print(f"No data found for object ID: {object_id}")
            return

        object_name = data.get('object', {}).get('fullname', 'Unknown')
        orbital_elements = data.get('orbit', {})
        physical_params = data.get('phys_par', {})
        
        print(f"\nObject Name: {object_name}")  
        print(f"Object Classification: {data.get('object', {}).get('class', 'Unknown')}")
        print(f"Discovery Date: {data.get('object', {}).get('discdate', 'Unknown')}")

        if orbital_elements:
            print("\nOrbital Elements:")
            print(f"Semi-Major Axis (AU): {orbital_elements.get('a', 'N/A')}")
            print(f"Eccentricity: {orbital_elements.get('e', 'N/A')}")
            print(f"Inclination (deg): {orbital_elements.get('i', 'N/A')}")

        if physical_params:
            print("\nPhysical Parameters:")
            print(f"Diameter (km): {physical_params.get('diameter', 'N/A')}")
            print(f"Mass (kg): {physical_params.get('mass', 'N/A')}")
            print(f"Rotation Period (hours): {physical_params.get('rot_per', 'N/A')}")

       

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching Solar System Dynamics data: {e}")
    except KeyError as e:
        print(f"Missing expected data in response: {e}")

def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()


def valid_email(email):
    return "@" in email and "." in email


def valid_password(password):
    special_characters = "!@#$%^&*()-+?_=,<>/"
    return len(password) >= 8 and any(char in special_characters for char in password)

def register():  
    print("Please Register")  
    name = input("Enter your name: ")  
    email = input("Enter your email: ")  
    password = input("Enter your password (at least 8 characters, with one special character): ")  
    security_question = input("Enter your pet's name for security question: ")  

    if not valid_email(email):  
        print("Invalid email format.")  
        return  
    
    if not valid_password(password):  
        print("Password must be at least 8 characters long and contain at least one special character.")  
        return  

    hashed_password = hash_password(password)  

    with open("12319247.csv", 'a') as file:  
        file.write(f"{name},{email},{hashed_password},{security_question}\n")  
    
    print("Registration Done")  

def login():  
    print("User Login")  
    max_attempts = 5   
    attempts = 0   
    email = input("Enter your email: ")  
    
    if not valid_email(email):  
        print("Invalid email format.")  
        return  

    while attempts < max_attempts:  
        password = input("Enter your password: ")  
        hashed_password = hash_password(password)  
    
        with open("12319247.csv", 'r') as file:  
            reader = csv.reader(file)  
            for row in reader:  
                name, stored_email, stored_password, _ = row  
                if email == stored_email and hashed_password == stored_password:  
                    print(f"Welcome {name}! You have successfully logged in.")
                    
                    
                    while True:
                        print("\n Data Fetch Menu ")
                        print("1. Fetch Near Earth Objects Data")
                        print("2. Fetch Solar System Object Data")
                        print("3. Logout")
                        choice = input("Choose an option: ")
                        if choice == '1':
                            display_near_earth_objects()
                        elif choice == '2':
                            display_solar_object_info()
                        elif choice == '3':
                            print("Logging out.")
                            return
                        else:
                            print("Invalid choice. Please try again.")
                    
        attempts += 1  
        print(f"Invalid email or password. You have {max_attempts - attempts} attempts left.")  

    print("Too many failed login attempts. Please try again later.")  
 

def forgot_password():  
    print("Forgot Password")  
    email = input("Enter your email: ")  

    users = []  
    found_user = False  
    with open("12319247.csv", 'r') as file:  
        reader = csv.reader(file)  
        for row in reader:  
            users.append(row)  

    for user in users:  
        name = user[0]  
        stored_email = user[1]  
        security_question = user[3]  

        if email == stored_email:  
            found_user = True  
            answer = input(f"Security Question - What is your pet's name? ")  
            if answer == security_question:  
                new_password = input(f"Hello {name}, enter your new password (at least 8 characters, with one special character): ")  

                if not is_valid_password(new_password):  
                    print("Password must be at least 8 characters long and contain at least one special character.")  
                    return  

                hashed_password = hash_password(new_password)  
                user[2] = hashed_password  
                
                with open("12319247.csv", 'a') as file:  
                    file.write(f"{name},{email},{hashed_password},{security_question}\n") 

                print(f"Password updated successfully for {name}.")  
                return  
            else:  
                print("Security question answer is incorrect.")  
                return  
    
    if not found_user:  
        print("Email not found.")  


def main():
    while True:
        print("\n Main Menu ")
        print("1. Register")
        print("2. Login")
        print("3. Forgot Password")
        print("4. Exit")
        choice = input("Choose an option: ")
        if choice == '1':
            register()
        elif choice == '2':
            login()
        elif choice == '3':
            forgot_password()
        elif choice == '4':
            print("Exiting the application.")
            break
        else:
            print("Invalid choice. Please try again.")


main()



 Main Menu 
1. Register
2. Login
3. Forgot Password
4. Exit


Choose an option:  1


Please Register


Enter your name:  nishu
Enter your email:  nimisha@gmail.com
Enter your password (at least 8 characters, with one special character):  Nishu13@
Enter your pet's name for security question:  nishu


Registration Done

 Main Menu 
1. Register
2. Login
3. Forgot Password
4. Exit


...

User Login


Enter your email:  nimisha@gmail.com
Enter your password:  Nishu13@


Welcome nishu! You have successfully logged in.

 Data Fetch Menu 
1. Fetch Near Earth Objects Data
2. Fetch Solar System Object Data
3. Logout


Choose an option:  1



Close approach date: 2024-10-09
Name: 534676 (2014 VK2)
Estimated Diameter (m): 494.94 - 1106.73
Velocity (km/h): 64945.5757870626
Miss Distance (km): 33576575.627546807
Hazardous: No
Name: (2018 QE)
Estimated Diameter (m): 7.67 - 17.14
Velocity (km/h): 15852.7439759616
Miss Distance (km): 668092.029831678
Hazardous: No
Name: (2019 OV3)
Estimated Diameter (m): 37.37 - 83.57
Velocity (km/h): 36178.0919025401
Miss Distance (km): 54787252.748386633
Hazardous: No
Name: (2021 AG6)
Estimated Diameter (m): 11.18 - 25.01
Velocity (km/h): 35433.5045918994
Miss Distance (km): 59292207.26663549
Hazardous: No
Name: (2021 GP3)
Estimated Diameter (m): 40.05 - 89.54
Velocity (km/h): 50605.5085163341
Miss Distance (km): 58597032.711609427
Hazardous: No
Name: (2022 WB1)
Estimated Diameter (m): 56.31 - 125.90
Velocity (km/h): 46661.7791260098
Miss Distance (km): 58162688.340998691
Hazardous: No
Name: (2023 HC3)
Estimated Diameter (m): 43.71 - 97.73
Velocity (km/h): 21655.3109477025
Miss Distance (km): 